This notebook does the following:

    generate random walk polymer chain
    local optimization using openbabel
    packing of multiple polymer chains into the simulation box
    Compute the system topology and read in the force field data
    write the data file for lammps run

Dependencies:

    pymatgen(latest development version from github)
    Polymers
    packmol

Required input files:

    structures of start, bulk and terminal monomer molecules
    appropriately formatted yaml forcefield data file

In [1]:
import numpy as np

from pymatgen import Molecule
from pymatgen.io.babel import BabelMolAdaptor

from polymers.core import Polymer
from polymers.packmol import PackmolRunner
from polymers.utils import get_topology

Set the start, bulk, and terminal molecules. Also set the forcefield species name mapping

In [2]:
# start molecule
peo_start = Molecule.from_file("PEOmonomer_start.xyz")
ff_map = ["C", "H", "H", "H", "O", "C", "H", "H"]
peo_start.add_site_property("ff_map", ff_map)
s_head = 0
s_tail = 5

# chain molecule
peo_bulk = Molecule.from_file("PEOmonomer_bulk.xyz")
ff_map = ["C", "H", "H", "O", "C", "H", "H"]
peo_bulk.add_site_property("ff_map", ff_map)
head = 0
tail = 4

# linear chain molecule (shortest repeat unit with correct connectivity between monomers)
peo_bulk_linear = Molecule.from_file("peo_bulk_linear.xyz")
ff_map = ["C", "H", "H", "O", "C", "H", "H", "C", "H", "H", "O", "C", "H", "H"]
peo_bulk_linear.add_site_property("ff_map", ff_map)
head_l = 0
tail_l = 11

# terminal molecule
peo_end = Molecule.from_file("PEOmonomer_end.xyz")
ff_map = ["C", "H", "H", "O", "C", "H", "H", "H"]
peo_end.add_site_property("ff_map", ff_map)
e_head = 0
e_tail = 4

print peo_bulk_linear

Full Formula (H8 C4 O2)
Reduced Formula: H4C2O
Charge = 0, Spin Mult = 1
Sites (14)
0 C     0.000000     0.000000     0.000000
1 H    -0.372119    -0.517286    -0.901654
2 H    -0.372119    -0.517286     0.901654
3 O     1.507500     0.000000     0.000000
4 C     1.944850    -1.342030     0.000000
5 H     1.572730    -1.859320    -0.901654
6 H     1.572730    -1.859320     0.901654
7 C     3.452300    -1.354920     0.000000
8 H     3.824420    -0.837637     0.901654
9 H     3.824420    -0.837637    -0.901654
10 O     3.878160    -2.700650     0.000000
11 C     5.289610    -2.712720     0.000000
12 H     5.661720    -2.195430     0.901654
13 H     5.661720    -2.195430    -0.901654


Create polymer via random walk

In [3]:
n_units = 100
link_distance = 1.0

# create the polymer
#peo_polymer = Polymer(peo_start, s_head, s_tail, 
                      #peo_bulk, head, tail, 
                      #peo_end, e_head, e_tail, 
                      #n_units, link_distance)

# linear chain
peo_polymer_linear = Polymer(peo_start, s_head, s_tail, 
                             peo_bulk_linear, head_l, tail_l, 
                             peo_end, e_head, e_tail, 
                             n_units, link_distance, linear_chain=True)

peo_polymer_linear.molecule.to(filename="polymer_linear.xyz", fmt="xyz")

Topology of the peo_polymer, from the linear chain

In [ ]:
from pymatgen.io.lammps.topology import Topology
topology = Topology.from_molecule(peo_polymer_linear.molecule)
print len(topology.atoms), len(topology.bonds), len(topology.angles), len(topology.dihedrals)

Use Packmol to pack the polymer chains into a box

In [ ]:
# one polymer molecule is fixed at the origin, surrounded by 5 polymer chains within a radius of 200A
packmol_config = [{"number": 1, "fixed": [0, 0, 0, 0, 0, 0],"centerofmass": ""},
                  {"number": 5, "inside sphere": [0, 0, 0, 100.0]}]
pmr = PackmolRunner([peo_polymer.molecule, peo_polymer.molecule],
                    packmol_config,
                    tolerance=2.0,
                    filetype="xyz",
                    control_params={"nloop": 1000},
                    auto_box=False, output_file="poly_packed.xyz")
packed_polymer = pmr.run()

Read in the force field parameters

In [ ]:
from pymatgen.io.lammps.force_field import ForceField
forcefield = ForceField.from_file("ff_data.yaml")

Generate lammps data file from the topology and the forcefield

In [ ]:
from pymatgen.io.lammps.data import LammpsForceFieldData

mols = [peo_polymer.molecule]*2
mols_number = [1, 5]
box_size = [[0.0, 300.0], [0.0, 300.0], [0.0, 300.0]]
molecule = packed_polymer
topologies = [topology]*2
lammps_ff_data = LammpsForceFieldData.from_forcefield_and_topology(mols, mols_number, 
                                                                   box_size, molecule, 
                                                                   forcefield, topologies)

Write the data file(uncomment the write line)

In [ ]:
print str(lammps_ff_data)

#write the data file
#lammps_ff_data.write_data_file("lammps_data.dat")